In [1]:
module CNN

const Lmax = 2 # maximum number of layers
const Nmax = 1000 # maximum number of neurons
const Nmin = 50 # minimum number of neurons
const Nstep = 50 # step of number of neurons
const Cmax = 100 # maximum number of channels
const Cmin = 10  # minimum number of channels
const Cstep = 10 # step of number of channels
const Wmax = 10  # maximum window size
const Wmin = 4   # minimum  window size 
const Wstep = 1  # step of  window size
const epochs = 10 # training epochs
const maxevals = 2 # minimum evaluations of optimization functions
const lr_min = 0.01 # minimum learning rate
const lr_max = 0.5 # maximum learning rate

using Knet, Hyperopt

function splitdata(x, y)
    n1, n2 = size(x)
    xtrn = x[:,1:4*n2÷5]
    xtst = x[:,4*n2÷5+1:n2]
    ytrn = y[:,1:4*n2÷5]
    ytst = y[:,4*n2÷5+1:n2]
    return  xtrn, ytrn, xtst, ytst
end 

function label2vec(l)
    global lu
    lu = unique(l) # unique label
    eltype(lu)<:Number && (lu = sort(lu))
    v = zeros(length(lu),length(l))
    for j = 1:size(v,2)
        i = find(lu .== l[j])[1]
        v[i,j] = 1
    end
    v
end

function vec2label(v)
    global lu
    l = Array{eltype(lu)}(1,size(v,2))
    for j = 1:size(v,2)
        i = findmax(v[:,j])[2]
        l[1,j] = lu[i]
    end
    l
end

function preprocess(x,y)
    global M, m
    
    if size(x,1) > size(x,2)
        x = x'
    end
    M = maximum(x)
    m = minimum(x)
    x = (x.- m)./(M .- m .+ 1e-20)
    y = label2vec(y)
    return x,y
end

function xavier(a...)
    w = rand(a...)
    if ndims(w) < 2
        error("ndims=$(ndims(w)) in xavier")
    elseif ndims(w) == 2
        fanout = size(w,1)
        fanin = size(w,2)
    else
        fanout = size(w, ndims(w)) 
        fanin = div(length(w), fanout)
    end
    s = sqrt(2 / (fanin + fanout))
    w = 2s*w-s
end


function weights(data,W,O,H) 
    # W:windowsizes, O:outputchannels, H:hiddenlayers
    global atype
    x, y = data[1]
    w = [Any[],Any[]]
  
    I = zeros(O) # input channel
    I[1] = size(x,3)
    I[2:end] = O[1:end-1]   
    for i = 1:length(W)  
        push!(w[1], atype(xavier(Float32,W[i],W[i],I[i],O[i])))
        x = conv4(w[1][end],x)
        push!(w[1], atype(zeros(Float32,1,1,O[i],1)))
        x =  pool(relu(x.+ w[1][end]))
    end
    x = mat(x)
    
    push!(w[2], atype(xavier(Float32,H[1],size(x,1))))
    push!(w[2], atype(zeros(Float32,H[1],1)))
    for i = 2:length(H)
        push!(w[2], atype(xavier(Float32,H[i],H[i-1])))
        push!(w[2], atype(zeros(Float32,H[i],1)))
    end
    push!(w[2], atype(xavier(Float32,size(y,1),H[end])))
    push!(w[2], atype(zeros(Float32,size(y,1),1)))
    return w
    
end

function minibatch(x, y, batchsize; atype=Array{Float32})
    x = atype(x); y = atype(y)
    data = Any[]
    for i=1:batchsize:size(x,2)
        j=min(i+batchsize-1,size(x,2))
        push!(data, (x[:,i:j], y[:,i:j]))
    end
    return data
end

function minibatch4(x, y, batchsize, imagesize; atype=KnetArray{Float32})
    data = minibatch(x,y,batchsize; atype=atype)
    for i=1:length(data)
        (x,y) = data[i]
        data[i] = (reshape(x, (imagesize...,batchsize)), y)
    end
    return data
end


function predict(w,x)
    for i = 1:2:length(w[1])
        x = pool(relu(conv4(w[1][i],x) .+ w[1][i+1]))
    end
    x = mat(x)
    for i = 1:2:length(w[2])-2
        x = relu(w[2][i]*x .+ w[2][i+1])
    end
    return w[2][end-1]*x .+ w[2][end]
end

function accuracy(w, dtst)
    ncorrect = ninstance = nloss = 0
    for (x, ygold) in dtst
        ypred = predict(w, x)
        ynorm = ypred .- log(sum(exp(ypred),1))
        nloss += -sum(ygold .* ynorm)
        ncorrect += sum((ypred .== maximum(ypred,1)) .* (ygold .== maximum(ygold,1)))
        ninstance += size(ygold,2)
    end
    return (ncorrect/ninstance, nloss/ninstance)
end

function loss(w,x,ygold)
    ypred = predict(w,x)
    ynorm = logp(ypred,1) # ypred .- log(sum(exp(ypred),1))
    -sum(ygold .* ynorm) / size(ygold,2)
end

lossgradient = grad(loss)

function train(w, dtrn; lr=.5, epochs=10)
    for epoch=1:epochs
        for (x,y) in dtrn
            g = lossgradient(w, x, y) 
            for i in 1:length(w[1])
                w[1][i] -= lr * g[1][i]
            end                      
            for i in 1:length(w[2])
                w[2][i] -= lr * g[2][i]
            end            
        end
    end
    return w
end


function objective(args)
    H, C, lr = args
    H = Int.(collect(H))
    W = Int[x[1] for x in C]
    O = Int[x[2] for x in C]
        
    global dtrn, dtst
    w = weights(dtrn, W, O, H)
    
    loss = 1000
    for i=1:epochs
        train(w, dtrn; lr=lr, epochs=1)
        err = 1 - accuracy(w, dtst)[1]
        if err < loss
            loss = err
        end
    end
    @printf("\nWindowSize=%s,OutputChannels=%s,HiddenLayers=%s,lr=%6.4f,loss=%6.4f\n",
    W,O,H,lr,loss)
    
    return Dict("loss" => loss, "status" => STATUS_OK, "model" => w)
end

global dtrn, dtst, M, m, lu, atype

export main
function main(x, y, imagesize; batchsize=50, gpu=true)
    global dtrn, dtst, atype
    
    x, y = preprocess(x, y)
    xtrn, ytrn, xtst, ytst = splitdata(x, y)
    gpu == false ? (atype = Array{Float32}) : (atype = KnetArray{Float32})
    dtrn = minibatch4(xtrn, ytrn, batchsize, imagesize; atype=atype)
    dtst = minibatch4(xtst, ytst, batchsize, imagesize; atype=atype)
    
    trials = Trials()
    hps_H = [] # hyper-parameters of H(idden)
    opt_H = [] # option of H(idden)
    hps_C = [] # hyper-parameters of C(onv) = (W,O)
    opt_C = [] # option of C(onv) =(W,O)
    
    for l = 1:Lmax
        push!(hps_H,quniform("H$l",Nmin,Nmax,Nstep))
        push!(opt_H,(hps_H...))
        push!(hps_C,(quniform("W$l",Wmin,Wmax,Wstep),quniform("O$l",Cmin,Cmax,Cstep)))
        push!(opt_C,(hps_C...))
    end
    
    best_args = fmin(objective,
    space=[choice("hidden",opt_H), choice("conv",opt_C), uniform("lr", lr_min, lr_max)],
    algo=TPESUGGEST,
    maxevals=maxevals,
    trials = trials)
    
    best_loss, best_ind = findmin(losses(trials))    
    best_model = trials["results"][best_ind]["model"]
    
    best_model, best_args, best_loss
end

export predict
function predict(model::Array{Any}, x, ImageSize)
    global M, m, atype
    x = (x.- m)./(M .- m .+ 1e-20)
    y = Array(predict(model, atype(reshape(x, (ImageSize...,size(x,2))))))
    probs = exp(y)./sum(exp(y),1)
    preds = vec2label(y)
    return preds,probs
end 

end# end of module CNN


INFO: Knet using GPU 0


CNN

In [ ]:
import MNIST

x,y = MNIST.traindata()
x = x[:,1:1000]; y = y[1:1000]
net, net_args, net_loss = CNN.main(x,y,(28,28,1))
# net_preds, net_probs = CNN.predict(net, x[:,1:10], (28,28,1))

..................................................
WindowSize=[5],OutputChannels=[30],HiddenLayers=[100],lr=0.1513,loss=0.0303
.+


 in #conv4x#70(::Ptr{Void}, ::Float32, ::Float32, ::Int64, ::Ptr{Void}, ::Int64, ::Array{Any,1}, ::Function, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}) at /home/rluser/.julia/v0.5/Knet/src/cuda44.jl:50
 in #conv4x#85(::Array{Any,1}, ::Function, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::Knet.KnetArray{Float32,4}) at ./<missing>:0
 in #conv4#74(::Array{Any,1}, ::Function, ::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}) at ./<missing>:0
 in conv4(::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}) at ./<missing>:0
 in backward_pass(::AutoGrad.Rec{Array{Array{Any,1},1}}, ::AutoGrad.Rec{Float32}, ::Array{AutoGrad.Node,1}) at /home/rluser/.julia/v0.5/AutoGrad/src/cor


 in #conv4x#70(::Ptr{Void}, ::Float32, ::Float32, ::Int64, ::Ptr{Void}, ::Int64, ::Array{Any,1}, ::Function, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}) at /home/rluser/.julia/v0.5/Knet/src/cuda44.jl:50
 in #conv4x#85(::Array{Any,1}, ::Function, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::Knet.KnetArray{Float32,4}) at ./<missing>:0
 in #conv4#74(::Array{Any,1}, ::Function, ::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}) at ./<missing>:0
 in conv4(::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}) at ./<missing>:0
 in backward_pass(::AutoGrad.Rec{Array{Array{Any,1},1}}, ::AutoGrad.Rec{Float32}, ::Array{AutoGrad.Node,1}) at /home/rluser/.julia/v0.5/AutoGrad/src/cor

 in jl_Function_call(::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /home/rluser/.julia/v0.5/PyCall/src/callback.jl:35
 in macro expansion at /home/rluser/.julia/v0.5/PyCall/src/exception.jl:78 [inlined]
 in #_pycall#62(::Array{Any,1}, ::Function, ::PyCall.PyObject, ::Function, ::Vararg{Function,N}) at /home/rluser/.julia/v0.5/PyCall/src/PyCall.jl:551
 in (::PyCall.#kw##_pycall)(::Array{Any,1}, ::PyCall.#_pycall, ::PyCall.PyObject, ::Function, ::Vararg{Function,N}) at ./<missing>:0
 in #pycall#66(::Array{Any,1}, ::Function, ::PyCall.PyObject, ::Type{PyCall.PyAny}, ::Function, ::Vararg{Function,N}) at /home/rluser/.julia/v0.5/PyCall/src/PyCall.jl:568
 in (::PyCall.#kw##pycall)(::Array{Any,1}, ::PyCall.#pycall, ::PyCall.PyObject, ::Type{PyCall.PyAny}, ::Function, ::Vararg{Function,N}) at ./<missing>:0
 in #call#67(::Array{Any,1}, ::PyCall.PyObject, ::Function, ::Vararg{Function,N}) at /home/rluser/.julia/v0.5/PyCall/src/PyCall.jl:571
 in (

,Int64,Int64}) at ./In[1]:224
 in main(::Array{Float64,2}, ::Array{Float64,1}, ::Tuple{Int64,Int64,Int64}) at ./In[1]:203
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/rluser/.julia/v0.5/IJulia/src/execute_request.jl:175
 in eventloop(::ZMQ.Socket) at /home/rluser/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
 in #conv4x#70(::Ptr{Void}, ::Float32, ::Float32, ::Int64, ::Ptr{Void}, ::Int64, ::Array{Any,1}, ::Function, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}) at /home/rluser/.julia/v0.5/Knet/src/cuda44.jl:50
 in #conv4x#85(::Array{Any,1}, ::Function, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::Knet.KnetArray{Float32,4}) at ./<missing>:0
 in #conv4#74(::Array{Any,1}, ::Function, ::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4

...


 in #conv4x#70(::Ptr{Void}, ::Float32, ::Float32, ::Int64, ::Ptr{Void}, ::Int64, ::Array{Any,1}, ::Function, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}) at /home/rluser/.julia/v0.5/Knet/src/cuda44.jl:50
 in #conv4x#85(::Array{Any,1}, ::Function, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::Knet.KnetArray{Float32,4}) at ./<missing>:0
 in #conv4#74(::Array{Any,1}, ::Function, ::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}) at ./<missing>:0
 in conv4(::Type{AutoGrad.Grad{2}}, ::Knet.KnetArray{Float32,4}, ::Knet.KnetArray{Float32,4}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}, ::AutoGrad.Rec{Knet.KnetArray{Float32,4}}) at ./<missing>:0
 in backward_pass(::AutoGrad.Rec{Array{Array{Any,1},1}}, ::AutoGrad.Rec{Float32}, ::Array{AutoGrad.Node,1}) at /home/rluser/.julia/v0.5/AutoGrad/src/cor

Int64})(::Array{Any,1}, ::Function, ::Array{Array{Any,1},1}, ::Vararg{Any,N}) at /home/rluser/.julia/v0.5/AutoGrad/src/core.jl:47
 in (::AutoGrad.#gradfun#2)(::Array{Array{Any,1},1}, ::Vararg{Any,N}) at /home/rluser/.julia/v0.5/AutoGrad/src/core.jl:47
 in #train#3(::Float64, ::Int64, ::Function, ::Array{Array{Any,1},1}, ::Array{Any,1}) at ./In[1]:163
 in (::CNN.#kw##train)(::Array{Any,1}, ::CNN.#train, ::Array{Array{Any,1},1}, ::Array{Any,1}) at ./<missing>:0
 in objective(::Tuple{Tuple{Float64,Float64},Tuple{Tuple{Float64,Float64},Tuple{Float64,Float64}},Float64}) at ./In[1]:187
 in jl_Function_call(::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /home/rluser/.julia/v0.5/PyCall/src/callback.jl:35
 in macro expansion at /home/rluser/.julia/v0.5/PyCall/src/exception.jl:78 [inlined]
 in #_pycall#62(::Array{Any,1}, ::Function, ::PyCall.PyObject, ::Function, ::Vararg{Function,N}) at /home/rluser/.julia/v0.5/PyCall/src/PyCall.jl:551
 in (::PyC

#kw##train)(::Array{Any,1}, ::CNN.#train, ::Array{Array{Any,1},1}, ::Array{Any,1}) at ./<missing>:0
 in objective(::Tuple{Tuple{Float64,Float64},Tuple{Tuple{Float64,Float64},Tuple{Float64,Float64}},Float64}) at ./In[1]:187
 in jl_Function_call(::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /home/rluser/.julia/v0.5/PyCall/src/callback.jl:35
 in macro expansion at /home/rluser/.julia/v0.5/PyCall/src/exception.jl:78 [inlined]
 in #_pycall#62(::Array{Any,1}, ::Function, ::PyCall.PyObject, ::Function, ::Vararg{Function,N}) at /home/rluser/.julia/v0.5/PyCall/src/PyCall.jl:551
 in (::PyCall.#kw##_pycall)(::Array{Any,1}, ::PyCall.#_pycall, ::PyCall.PyObject, ::Function, ::Vararg{Function,N}) at ./<missing>:0
 in #pycall#66(::Array{Any,1}, ::Function, ::PyCall.PyObject, ::Type{PyCall.PyAny}, ::Function, ::Vararg{Function,N}) at /home/rluser/.julia/v0.5/PyCall/src/PyCall.jl:568
 in (::PyCall.#kw##pycall)(::Array{Any,1}, ::PyCall.#pycall, ::PyCall.

Excessive output truncated after 524296 bytes.


 in 

In [ ]:
net